# Cryptography and Information Security 
## Symmetric Block Cipher assignment

- Fathi Al Adha Hylmi (22/492195/PA/21008)
- Louis Widi Anandaputra(22/492218/PA/21090)

We well be constructing a symmetric block cipher for a text message. This method would also work with other data, but we will be implementing it on encrypting strings using the CBC structure (for more than 1 block data) in a feistel structure which uses a finite field arithmetic (addition operation) for a Galois Field(GF).

1 block wold contain 2^32 data and the finite field arithmetic would use a GF(2^16). The feistel structure rounds can be set as a parameter but we will be using a 16 rounds feistel structure. We also define several functions:

- initialGenKey(length) --> intializing keys for both the key and initial vector (IV). Taking an integer parameter length (bytes)
- subGenKey(key) --> creating subkeys for the feistel cipher encryption process. Taking an integer parameter (key)
- reverseSubGenKey(last_key) --> reversing back the subkeys to the original key. Taking an integer parameter (last_key)
- GFaddition(plain, key) --> Finite Field addition on GF(2^16). Taking integer parameters (plain) and (key)
<br><br>
- encrypt_messageCBC(blockList, key, iv, len_of_block, round) --> CBC structure encryption for all blocks. Taking string array parameter (blockList), bytearray parameters (key) and (iv), and integer parameters (len_of_block) and (round)
- encrypt(plain_block, key) --> initial encryption with IV. Taking string parameter (plain_block) and bytearray parameter (key)
- feistelEncrypt(plain, key, round) --> feistel structure block encryption. Taking bytearray parameter (plain) and (key), and taking integer parameter (round)
<br><br>
- decrypt_cipherCBC(list_cipher, key, iv, round) --> CBC structure decryption for all blocks. Taking bytearray parameters (list_cipher) and (iv), and taking integer parameters (key, round)
- feistelDecrypt(cipher, key_int, round) --> feistel structure block decryption. Taking bytearray parameter (list) and integer parameters (key_int) and (round)
- decrypt(cipher, key) --> decryption with IV
<br><br>
- breakMessage(message, len_of_blocks) --> breaking data into blocks. Taking string parameter (message) and taking integer parameters (len_of_blocks)

### reference for code: https://www.youtube.com/watch?v=wwTsRONdAaw
<br><br>
<img src = 'https://ctf-wiki.mahaloz.re/crypto/blockcipher/mode/figure/cbc_encryption.png'> <img src = 'https://upload.wikimedia.org/wikipedia/commons/thumb/f/fa/Feistel_cipher_diagram_en.svg/300px-Feistel_cipher_diagram_en.svg.png'>

## Keys and Finite Field Arithmetic

In [1]:
from os import urandom

# Initial key generator
def initialGenKey(length):
    return bytearray(urandom(length))
    
def subGenKey(key):
    # perform an addition by 1 and XOR by 1 and 0
    value = (key + 1) ^ 0x10
    # reset the value to 0 if it exceeds the maximum limit
    shifted_key = value & ((1 << 16) - 1)  # 16 bit limit
    print(f"Initial key: {key}; new key: {shifted_key}")
    return shifted_key
    
def reverseSubGenKey(last_key):
    # perform the inverse XOR operation to recover the value before XOR
    value = (last_key ^ 0x10)-1
    # reset the value to 0 if it exceeds the maximum limit
    key = value & ((1 << 16) - 1)# 16 bit limit
    print(f"Initial key: {last_key}; new key: {key}")
    return key
    
# we will be using a 2^16 finite field addition operation, which will be using XOR operation
def GFaddition(plain, key):
    value = plain ^ key
    value = value & ((1 << 16) - 1)
    return value

## Encryption

In [2]:
def encrypt_messageCBC(blockList, key, iv, len_of_block, round):
    list_of_ciphers = []
    for i in range(len(blockList)):
        print (f"\nBlock {i}:\n{blockList[i]}\n")
        # performing initial encryption with initial vector
        cipher1 = encrypt(blockList[i], iv)
        # performing the feistel cipher encryption
        cipher2, key_last = feistelEncrypt(cipher1, key, round)
        list_of_ciphers.append(cipher2)
        iv = cipher2
    return list_of_ciphers, key_last

In [3]:
def encrypt(plain_block, key):
    return bytearray([ord(plain_block[i]) ^ key[i] for i in range(len(plain_block))])

In [4]:
def feistelEncrypt(plain, key, round):
    # Changing key and plaintext into integers for encryption process
    key_int = int.from_bytes(key, byteorder='big')
    plain = int.from_bytes(plain, byteorder='big')

    # Splitting the bits to half (16 - 16)
    left = plain >> 16
    right = plain & 0xffff
    print(f"Initial left partition: {left}\nInitial right partition: {right}\n")

    # feistel structure
    for k in range ( round):  #rounds
        right = GFaddition(right, key_int) # Using the Galois Field encryption
        left = left ^ right
        print(f"Round {k+1}:Round {k+1}:\n right partition(after applied F, Key:{key_int}): {right}\n left partition(after XOR with right, right:{right}): {left}")

        # generating new subkey
        key_int = subGenKey(key_int)

        #switching the left and right data
        temp = right
        right = left
        left = temp
        print(f"Round {k+1} switching results:\n left partition: {left}\n right partition: {right}\n")

    # final switch for the left and right data   
    temp = right
    right = left
    left = temp    
    print(f"Final left partition: {left}\nInitial right partition: {right}\n")
    
    key = key_int
    # making sure the length is still 2^16
    encrypted_block = (left <<16)|right
    # changing back into bytes
    encrypted_block = encrypted_block.to_bytes((encrypted_block.bit_length() + 7) // 8, byteorder='big')
    return encrypted_block, key

## Decryption

In [5]:
def decrypt_cipherCBC(list_cipher, key, iv, round):
    list_of_decrypted_blocks = []
    temp, key_last = feistelDecrypt(list_cipher[0], key, round)
    plain = decrypt(temp, iv)
    list_of_decrypted_blocks.append(plain)
    for i in range(1, len(list_cipher)):
        temp, key_last = feistelDecrypt(list_cipher[i], key, round)
        plain = decrypt(temp, list_cipher[i-1])
        list_of_decrypted_blocks.append(plain)
    return list_of_decrypted_blocks


In [6]:
# Change this according to the feistel encryption method
def feistelDecrypt(cipher, key_int, round):
    cipher = int.from_bytes(cipher, byteorder='big')
    key_int = reverseSubGenKey(key_int)
    left = cipher >> 16
    right = cipher & 0xffff
    print(f"Initial left partition: {left}\nInitial right partition: {right}\n")
  
    for k in range (round):  #rounds
        left = left ^ right
        right = GFaddition(right, key_int)
        
        print(f"Round {k+1}:\n right partition(after applied F, Key:{key_int}): {right}\n left partition(after XOR with right, right:{right}): {left}")
        
        key_int = reverseSubGenKey(key_int)
        temp = right
        right = left
        left = temp
        print(f"Round {k+1} switching results:\n left partition: {left}\n right partition: {right}\n")

    temp = right
    right = left
    left = temp
    print(f"Final left partition: {left}\nInitial right partition: {right}\n")
    
    key = key_int
    decrypted_block = (left <<16)|right
    decrypted_block = decrypted_block.to_bytes((decrypted_block.bit_length() + 7) // 8, byteorder='big')
    return decrypted_block, key

In [7]:
# Change this according to the feistel encryption method
def decrypt(cipher, key):
    return [chr(cipher[i] ^ key[i]) for i in range(len(cipher))]

## Block Creation

In [8]:
def breakMessage(message, len_of_blocks):
    list_of_blocks = []
    for i in range (0, len(message), len_of_blocks):
        block = message[i:i+len_of_block]
        if(len(block) == len_of_blocks):
            list_of_blocks.append(block)
        else:
            c = len_of_blocks - len(block)
            for i in range(c):
                block = block + " "
            list_of_blocks.append(block)
    return list_of_blocks
                    

## Demo

In [9]:
# Initialization
text = "Hello world! This is Hylmi and Louis"
len_of_block = 4
len_of_feistel = 2
round = 16
blockList = breakMessage(text, len_of_block)

In [10]:
key = initialGenKey(len_of_feistel)
iv = initialGenKey(len_of_block)


In [11]:
print(f"Key: {key}\nIV: {iv}")

Key: bytearray(b'E\xd4')
IV: bytearray(b'vF\x1dK')


In [12]:
# encryption
cipher_list, key_last_used= encrypt_messageCBC(blockList, key, iv, len_of_block, round = round)


Block 0:
Hell

Initial left partition: 15907
Initial right partition: 28967

Round 1:Round 1:
 right partition(after applied F, Key:17876): 13555
 left partition(after XOR with right, right:13555): 2768
Initial key: 17876; new key: 17861
Round 1 switching results:
 left partition: 13555
 right partition: 2768

Round 2:Round 2:
 right partition(after applied F, Key:17861): 20245
 left partition(after XOR with right, right:20245): 31718
Initial key: 17861; new key: 17878
Round 2 switching results:
 left partition: 20245
 right partition: 31718

Round 3:Round 3:
 right partition(after applied F, Key:17878): 15920
 left partition(after XOR with right, right:15920): 28965
Initial key: 17878; new key: 17863
Round 3 switching results:
 left partition: 15920
 right partition: 28965

Round 4:Round 4:
 right partition(after applied F, Key:17863): 13538
 left partition(after XOR with right, right:13538): 2770
Initial key: 17863; new key: 17880
Round 4 switching results:
 left partition: 13538
 r

In [13]:
# decryption
decrypted_list = decrypt_cipherCBC(cipher_list, key_last_used, iv, round)

Initial key: 17860; new key: 17875
Initial left partition: 2762
Initial right partition: 13566

Round 1:
 right partition(after applied F, Key:17875): 28973
 left partition(after XOR with right, right:28973): 15924
Initial key: 17875; new key: 17858
Round 1 switching results:
 left partition: 28973
 right partition: 15924

Round 2:
 right partition(after applied F, Key:17858): 31734
 left partition(after XOR with right, right:31734): 20249
Initial key: 17858; new key: 17873
Round 2 switching results:
 left partition: 31734
 right partition: 20249

Round 3:
 right partition(after applied F, Key:17873): 2760
 left partition(after XOR with right, right:2760): 13551
Initial key: 17873; new key: 17856
Round 3 switching results:
 left partition: 2760
 right partition: 13551

Round 4:
 right partition(after applied F, Key:17856): 28975
 left partition(after XOR with right, right:28975): 15911
Initial key: 17856; new key: 17871
Round 4 switching results:
 left partition: 28975
 right partition

### Results

In [14]:
# plaintext
for i in blockList:
    print ([(i[k]) for k in range(len(i))])

['H', 'e', 'l', 'l']
['o', ' ', 'w', 'o']
['r', 'l', 'd', '!']
[' ', 'T', 'h', 'i']
['s', ' ', 'i', 's']
[' ', 'H', 'y', 'l']
['m', 'i', ' ', 'a']
['n', 'd', ' ', 'L']
['o', 'u', 'i', 's']


In [15]:
# ciphertext
for i in cipher_list:
    print ([chr(i[k]) for k in range(len(i))])

['\n', 'Ê', '4', 'þ']
['c', 'µ', '\x06', 'H']
['6', '~', "'", '°']
['\x1c', '=', '\n', '\x00']
['I', '\xa0', '&', 'ª']
['s', 'à', '\x1a', '\x1f']
['a', '9', '\x7f', '§']
['\x15', 'x', '\x1a', '2']
['L', '\x82', '6', '\x98']


In [16]:
# plaintext
for i in decrypted_list:
    print ([(i[k]) for k in range(len(i))])

['H', 'e', 'l', 'l']
['o', ' ', 'w', 'o']
['r', 'l', 'd', '!']
[' ', 'T', 'h', 'i']
['s', ' ', 'i', 's']
[' ', 'H', 'y', 'l']
['m', 'i', ' ', 'a']
['n', 'd', ' ', 'L']
['o', 'u', 'i', 's']


© 2024 - Fathi Al Adha Hylmi & Louis Widi Anandaputra